In [1]:
import pandas as pd 
import numpy as np
from ast import literal_eval
import webbrowser
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

## Data preprocessing and exploration

In [2]:
#reading the datasets into dataframes
df_creds=pd.read_csv('tmdb_5000_credits.csv')
df_details=pd.read_csv('tmdb_5000_movies.csv')

In [3]:
print(df_creds.shape)
print(df_details.shape)

(4803, 4)
(4803, 20)


In [4]:
df_creds.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
df_details.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
#combining the 2 files into one dataframe
df_creds.columns = ['id','tittle','cast','crew']
df_main= df_details.merge(df_creds,on='id')

In [7]:
df_main.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [8]:
#checking the data fields
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [9]:
#check for null values
df_main.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
tittle                     0
cast                       0
crew                       0
dtype: int64

In [10]:
#removing the column 'tittle' which is redundant after merging the datasets
df_main.drop("tittle",axis=1,inplace=True)

In [11]:
#checking for any duplicates
df_main.duplicated().sum()

0

## Recommender System

##### Calculation of a weighted rating based on the IMDB formula/credibility formula

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where:

    W is the weighted rating
    R is the mean rating for the movie(vote_average), from 1 to 10
    v is the number of votes for the movie(vote_count)
    m is the minimum votes required (95th percentile)
    C is the mean vote across the whole report

In [12]:
C = df_main['vote_average'].mean()
m = df_main['vote_count'].quantile(0.9)

#filtered list of movies based on percentile
df_gen = df_main.copy().loc[df_main['vote_count'] >= m]

#function based on the above referenced formula
def rating_w(df, m=m, C=C):
    v = df['vote_count']
    R = df['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [13]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
df_gen['score'] = df_gen.apply(rating_w, axis=1)

In [14]:
#Sort movies based on score calculated above
df_gen = df_gen.sort_values('score', ascending=False)
df_gen.reset_index(drop=True, inplace=True)
df_gen.index = df_gen.index + 1

def generic_rec(count):
    return (df_gen[['title', 'vote_count', 'vote_average', 'score','id']].head(count))

In [15]:
def generic_rec_genre(genre,count):
    df_genre = df_gen[df_gen['genres'].str.contains(genre)][['title', 'vote_count', 'vote_average', 'score','id']].head(count)
    df_genre.reset_index(drop=True, inplace=True)
    df_genre.index = df_genre.index + 1
    return df_genre

In [16]:
df_disp = generic_rec_genre("Adventure",10)

In [17]:
def more_details(df=df_disp):
    top = df['title'].values[0]
    det = input('Please enter the index of the movie, for example, enter \'1\' for \''+top+'\': \n \n')
    ind = int(det)-1
    name = df['title'].values[ind]
    name = name.replace(" ","-")
    movie_id = df['id'].values[ind]
    url = "https://www.themoviedb.org/movie/"+str(movie_id)+'-'+name
    
    webbrowser.open(url)

In [18]:
df_disp[['title', 'vote_count', 'vote_average', 'score']]

,title,vote_count,vote_average,score
1,Inception,13752,8.1,7.863239
2,Interstellar,10867,8.1,7.809479
3,The Lord of the Rings: The Return of the King,8064,8.1,7.727243
4,The Empire Strikes Back,5879,8.2,7.697884
5,The Lord of the Rings: The Fellowship of the Ring,8705,8.0,7.667341
6,Star Wars,6624,8.1,7.663813
7,The Lord of the Rings: The Two Towers,7487,8.0,7.623893
8,Guardians of the Galaxy,9742,7.9,7.613005
9,Spirited Away,3840,8.3,7.585209
10,Back to the Future,6079,8.0,7.557007


In [19]:
# more_details()

In [20]:
# Parse the stringified features into their corresponding python objects
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df_main[feature] = df_main[feature].apply(literal_eval)

In [21]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [22]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [23]:
# Define new director, cast, genres and keywords features that are in a suitable form.
df_main['director'] = df_main['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    df_main[feature] = df_main[feature].apply(get_list)

In [24]:
# Print the new features of the first 3 films
df_main[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"


In [25]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [26]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    df_main[feature] = df_main[feature].apply(clean_data)

In [27]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df_main['soup'] = df_main.apply(create_soup, axis=1)

In [28]:
# Creating the count matrix
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_main['soup'])

In [29]:
# Compute the Cosine Similarity matrix based on the count_matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [30]:
df_main['tagline'] = df_main['tagline'].fillna('')
df_main['description'] = df_main['overview'] + df_main['tagline']
df_main['description'] = df_main['description'].fillna('')

In [31]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_main['description'])

In [32]:
cosine_sim_2 = linear_kernel(tfidf_matrix, tfidf_matrix)

In [33]:
# Reset index of our main DataFrame and construct reverse mapping as before
df_main = df_main.reset_index()
indices = pd.Series(df_main.index, index=df_main['title'])

In [34]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim, cosine_sim_2=cosine_sim_2):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores_2 = list(enumerate(cosine_sim_2[idx]))
    
    x=(sim_scores,sim_scores_2)
    
    from numpy import mean
    sim_scores_avg = tuple(mean(x, axis=0))
    
    # Sort the movies based on the similarity scores
    sim_scores_avg = sorted(sim_scores_avg, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores_avg = sim_scores_avg[1:21]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores_avg]

    # Return the top 10 most similar movies
    return df_main['title'].iloc[movie_indices]

In [35]:
get_recommendations('The Dark Knight')

3                 The Dark Knight Rises
119                       Batman Begins
4638           Amidst the Devil's Wings
2398                             Hitman
299                      Batman Forever
1720                           Kick-Ass
1740                         Kick-Ass 2
3326                     Black November
1503                             Takers
1986                             Faster
3725                        The Sweeney
1253                      Kiss of Death
1278                         The Gunman
4135    Gangster's Paradise: Jerusalema
303                            Catwoman
747                      Gangster Squad
2154                       Street Kings
2793               The Killer Inside Me
4664                            Bronson
3062                        Deuces Wild
Name: title, dtype: object

In [36]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4803 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4803 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [ ]:
def get_rec_user(df=df_main):
    
    df_main['description'] = df_main['overview'] + df_main['tagline'] + df_main['genres'].astype(str) + df_main['keywords'].astype(str)
    df_main['description'] = df_main['description'].fillna('')
    
    m = df_main['vote_count'].quantile(0.4)

    #filtered list of movies based on percentile
    df_main_copy = df_main.copy().loc[df_main['vote_count'] >= m]
    
    user_in = input("Enter search term here: ")
    
    df_main_copy.loc[len(df_main_copy.index)] = ["","","","","","","","","","","","","","","","","","","","","","","","","",user_in]
    
    tfidf_matrix = tf.fit_transform(df_main_copy['description'])
    
    cosine_sim_3 = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    # Get the index of the movie that matches the title
    idx = len(df_main_copy.index)-1

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim_3[idx]))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df_main_copy['title'].iloc[movie_indices]

In [ ]:
get_rec_user()